In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

model = ChatOpenAI(
    model = "gpt-3.5-turbo",
    max_completion_tokens=100
)

In [3]:
class BlogState(TypedDict) :
    title: str
    outline: str
    content: str

In [4]:

def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [5]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [6]:

graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [7]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': 'I. Introduction\n    A. Explanation of AI (Artificial Intelligence) and its importance\n    B. Overview of the rise of AI globally\n    C. Introduction to the rise of AI in India\n\nII. History of AI in India\n    A. Early developments in AI technology in India\n    B. Government initiatives to promote AI research and development\n    C. Growth of AI startups and companies in India\n\nIII. Applications of AI in India\n    A. AI in healthcare\n', 'content': '   B. AI in agriculture\n   C. AI in education\n   D. AI in finance\n   E. AI in customer service\n\nIV. Challenges faced by AI in India\n    A. Lack of skilled professionals\n    B. Data privacy and security concerns\n    C. Ethical considerations\n    D. Infrastructure limitations\n\nV. Future prospects of AI in India\n    A. Potential for economic growth and innovation\n    B. Role of AI in addressing societal challenges\n    C'}


In [8]:
print(final_state['outline'])

I. Introduction
    A. Explanation of AI (Artificial Intelligence) and its importance
    B. Overview of the rise of AI globally
    C. Introduction to the rise of AI in India

II. History of AI in India
    A. Early developments in AI technology in India
    B. Government initiatives to promote AI research and development
    C. Growth of AI startups and companies in India

III. Applications of AI in India
    A. AI in healthcare



In [9]:
print(final_state['content'])

   B. AI in agriculture
   C. AI in education
   D. AI in finance
   E. AI in customer service

IV. Challenges faced by AI in India
    A. Lack of skilled professionals
    B. Data privacy and security concerns
    C. Ethical considerations
    D. Infrastructure limitations

V. Future prospects of AI in India
    A. Potential for economic growth and innovation
    B. Role of AI in addressing societal challenges
    C
